In [69]:
# importing libraries

from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.agents import *
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities import SQLDatabase
from langchain import HuggingFacePipeline
from sqlalchemy import text
import pandas as pd
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import huggingface_hub
from langchain import HuggingFaceHub
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
from langchain.prompts.prompt import PromptTemplate

In [70]:
# connect to datatbase
load_dotenv()


# Access environment variables
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_NAME2 = os.getenv("DB_NAME2")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
HF_KEY= os.getenv("HUGGINGFACEHUB_API_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from huggingface_hub import login
login(token=HF_KEY)


db = SQLDatabase.from_uri(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
db2 = SQLDatabase.from_uri(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME2}")
# engine = sqla.create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/chelciedealmeida/.cache/huggingface/token
Login successful


/usr/local/lib/python3.9/site-packages/langchain/utilities/sql_database.py:111: SAWarning: Did not recognize type 'point' of column 'coordinates'
  self._metadata.reflect(


In [121]:
# Setting up llm

modelName="gpt-3.5-turbo-16k" # replace value with model of choice
llm = ChatOpenAI(model_name=modelName, temperature=0.2) # change between 0 and 0.2 for testing

In [122]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm) # change db to db2 if you want to test another database
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [ ]:


# _DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.

# Use the following format:

# Question: "Question here"
# SQLQuery: "SQL Query to run"
# SQLResult: "Result of the SQLQuery"
# Answer: "Final answer here"

# The SQL query should NOT end with semi-colon
# Question: {input}"""

# PROMPT = PromptTemplate(
#      input_variables=["input", "dialect"], template=_DEFAULT_TEMPLATE
# )


# chain = SQLDatabaseChain(llm=llm, database=db, prompt=PROMPT, verbose=True) 

## Testing Langchain on the Airport/bookings data

In [73]:
agent_executor.run("describe the bookings table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: 
Thought:I can see that there is a "bookings" table in the database. I should query the schema of this table to get more information.
Action: sql_db_schema
Action Input: "bookings"
Observation: Error: table_names {'bookings'} not found in database
Thought:

KeyboardInterrupt: 

In [ ]:
agent_executor.run("describe the flights table?")

In [ ]:
# TC1: PASS
agent_executor.run("How many rows are in the bookings table?")

In [ ]:
# TC2: PASS
agent_executor.run("How many rows are in the aircrafts_data table?")

In [ ]:
# TC3: PASS
agent_executor.run("find the top ten most expensive flights and order total amounts in descending order")

In [ ]:
# Testing for case sensitivity in values. Passenger name ANTONINA KUZNECOVA is spelled using the same case sensitivity as in the db
# executor correctly identified table and displayed the correct result based on the validation found in the validation_airports file.

#TC4: PASS
agent_executor.run("How many ticket bookings does passenger ANTONINA KUZNECOVA have? ")

In [ ]:

# langchain identified the value when the same case sensitivity used in the db was used in the prompt
# but failed to exexute query once the case sensitivity was incorrect. It did, however identify one of
# the correct tables {tickets} but joined to the {bookings} table which isn't incorrect. Executor did use case sensitivity 
# as instructed by the prompt.

# TC5: PASS 
agent_executor.run("How many ticket bookings does passenger Antonina Kuznecova have? ")

In [ ]:
# TC6: FAIL (it did manage to choose the correct table and identiy DME as russian airport but missed other values)
agent_executor.run("How many distinct flights departed from Moscow?Use the routes view to answer the question")

In [ ]:
# TC7: PASS
agent_executor.run("Get me the count of each flight status grouped by status and also provide the minimum and maximum departure ordered by the minimum scheduled departure ")

In [ ]:
#TC8 : PASS
agent_executor.run("Get me the count of each flight status grouped by status. Return only the status and count for each status type")

In [ ]:
#TC9 : PASS
agent_executor.run("Get me the list of seats and the fare conditions or class where those seats are for aircraft model Cessna 208 Caravan and output the aircraft code, the model, the seat number, and the fare conditions and order it by the seat number")

In [ ]:
#test
agent_executor.run("describe the ticket_flights table")

In [ ]:
#TC10 : FAIL
agent_executor.run("Get me the passenger name that had the most flights where the flight had a status of Arrived or Departed. The output should return the passenger name and a count of how many distinct flights this passenger had ")

In [ ]:
# test
agent_executor.run("describe the tickets table")

In [ ]:
# test
agent_executor.run("describe the ticket_flights table")

In [ ]:
#TC11 : FAIL
agent_executor.run("Get me the the passenger name, phone number, and email of 10 passengers ordered by passenger name in ascending order. Make sure to only retrieve data of passengers that have an email")

In [ ]:
#TC12 : FAIL
agent_executor.run("Which cities have more than 1 airport? Return the airport code, airport name, and city in your response.")

In [ ]:
# TC13 : PASS (With exception: didn't include 4 of the columns asked in the prompt) / temperature = 0
# TC13 : PASS (Perfect execution and included all of the required columns) / Temperature = 0.2
agent_executor.run("Get me the ticket number, flight id ,fare conditions, seat number, boarding number, passenger name, and amount for flight 4686 where the status is Arrived. Order the results by amount in descending order")

In [ ]:
#TC14 : PASS (with exception: it didn't extract the appropriate values from city columns which have data types of jsonb)
agent_executor.run("Which flight segments are included into ticket number 0005432661915? Please include the scheduled departure date only, the departure city along with the departure airport the arrival city along with the arrival airport, the status of the flight, and the seat numbers for each flight segment. Please output order by scheduled departure")

In [ ]:
# TC15 : FAIL (this test case is similar to TC14 which it passed with some exceptions. It tried to reference coordinates column which it has trouble manipulating. Still this approach was incorrect)
agent_executor.run("Which flight segments are included into ticket number 0005432661915? Please include the scheduled departure date only, the departure city along with the departure airport the arrival city along with the arrival airport, the fare conditions or class, and the amount each transaction cost. Please order by scheduled departure")

In [ ]:
# TC16 : PASS 
agent_executor.run("Get me the aircraft code and the fare conditions along with each fare conditions count for each aircraft code. For example, aircraft code 319 would have a fare conditions of business(20), Economy(96) where 20 and 96 represent the number of seats in economy fare conditions while 20 represents 20 business seats. The output should have the aircraft_code, and fare_conditions with their seat count in the same record. ")

## Testing Langchain on the AdventureWorksProd database

In [124]:
from langchain.chains.sql_database.base import SQLDatabaseChain
chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [125]:
chain.run("describe the employee table")



> Entering new SQLDatabaseChain chain...
describe the employee table
SQLQuery:SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name = 'employee';
SQLResult: [('businessentityid', 'integer'), ('salariedflag', 'boolean'), ('vacationhours', 'smallint'), ('sickleavehours', 'smallint'), ('currentflag', 'boolean'), ('rowguid', 'uuid'), ('modifieddate', 'timestamp without time zone'), ('birthdate', 'date'), ('hiredate', 'date'), ('nationalidnumber', 'character varying'), ('loginid', 'character varying'), ('jobtitle', 'character varying'), ('organizationnode', 'character varying'), ('maritalstatus', 'character'), ('gender', 'character')]
Answer:The employee table has the following columns:
- businessentityid (integer)
- salariedflag (boolean)
- vacationhours (smallint)
- sickleavehours (smallint)
- currentflag (boolean)
- rowguid (uuid)
- modifieddate (timestamp without time zone)
- birthdate (date)
- hiredate (date)
- nationalidnumber (character varying)
- loginid

'The employee table has the following columns:\n- businessentityid (integer)\n- salariedflag (boolean)\n- vacationhours (smallint)\n- sickleavehours (smallint)\n- currentflag (boolean)\n- rowguid (uuid)\n- modifieddate (timestamp without time zone)\n- birthdate (date)\n- hiredate (date)\n- nationalidnumber (character varying)\n- loginid (character varying)\n- jobtitle (character varying)\n- organizationnode (character varying)\n- maritalstatus (character)\n- gender (character)'

In [126]:
# TC1 : (temperature = 0)  | PASS
# TC1 : (temperature = 0.2)  | PASS
chain.run("How many rows are in the prod.employee table? Please do not reference humanresources.employee table. That schema no longer exists")



> Entering new SQLDatabaseChain chain...
How many rows are in the prod.employee table? Please do not reference humanresources.employee table. That schema no longer exists
SQLQuery:SELECT COUNT(*) FROM prod.employee;
SQLResult: [(290,)]
Answer:Final answer here: There are 290 rows in the prod.employee table.
> Finished chain.


'Final answer here: There are 290 rows in the prod.employee table.'

In [127]:
# TC2 : (temperature = 0)  | PASS
# TC2 : (temperature = 0.2)  | PASS
chain.run("Get me the jobtitle, maritalstatus, gender, and vacationhours of the top 5 employee with the most vacation hours from prod.employee")



> Entering new SQLDatabaseChain chain...
Get me the jobtitle, maritalstatus, gender, and vacationhours of the top 5 employee with the most vacation hours from prod.employee
SQLQuery:SELECT "jobtitle", "maritalstatus", "gender", "vacationhours"
FROM prod.employee
ORDER BY "vacationhours" DESC
LIMIT 5;
SQLResult: [('Production Technician - WC50', 'M', 'M', 99), ('Production Technician - WC10', 'S', 'F', 99), ('Chief Executive Officer', 'S', 'M', 99), ('Production Technician - WC10', 'M', 'M', 98), ('Production Technician - WC50', 'S', 'M', 98)]
Answer:The jobtitle, maritalstatus, gender, and vacationhours of the top 5 employees with the most vacation hours from prod.employee are:

1. Job Title: Production Technician - WC50, Marital Status: M, Gender: M, Vacation Hours: 99
2. Job Title: Production Technician - WC10, Marital Status: S, Gender: F, Vacation Hours: 99
3. Job Title: Chief Executive Officer, Marital Status: S, Gender: M, Vacation Hours: 99
4. Job Title: Production Technician 

'The jobtitle, maritalstatus, gender, and vacationhours of the top 5 employees with the most vacation hours from prod.employee are:\n\n1. Job Title: Production Technician - WC50, Marital Status: M, Gender: M, Vacation Hours: 99\n2. Job Title: Production Technician - WC10, Marital Status: S, Gender: F, Vacation Hours: 99\n3. Job Title: Chief Executive Officer, Marital Status: S, Gender: M, Vacation Hours: 99\n4. Job Title: Production Technician - WC10, Marital Status: M, Gender: M, Vacation Hours: 98\n5. Job Title: Production Technician - WC50, Marital Status: S, Gender: M, Vacation Hours: 98'

In [128]:
# TC3 : (temperature = 0) |  PASS
# TC3 : (temperature = 0.2) |  PASS
chain.run("Get me the jobtitle, maritalstatus, gender, and vacationhours of the top 5 employees with the least vacation hours from prod.employee")



> Entering new SQLDatabaseChain chain...
Get me the jobtitle, maritalstatus, gender, and vacationhours of the top 5 employees with the least vacation hours from prod.employee
SQLQuery:SELECT "jobtitle", "maritalstatus", "gender", "vacationhours"
FROM prod.employee
ORDER BY "vacationhours" ASC
LIMIT 5;
SQLResult: [('Chief Financial Officer', 'M', 'F', 0), ('Production Technician - WC20', 'S', 'M', 0), ('Production Technician - WC50', 'S', 'M', 0), ('Production Technician - WC50', 'M', 'M', 1), ('Vice President of Engineering', 'S', 'F', 1)]
Answer:The jobtitle, maritalstatus, gender, and vacationhours of the top 5 employees with the least vacation hours are:
1. Job Title: Chief Financial Officer, Marital Status: M, Gender: F, Vacation Hours: 0
2. Job Title: Production Technician - WC20, Marital Status: S, Gender: M, Vacation Hours: 0
3. Job Title: Production Technician - WC50, Marital Status: S, Gender: M, Vacation Hours: 0
4. Job Title: Production Technician - WC50, Marital Status: M

'The jobtitle, maritalstatus, gender, and vacationhours of the top 5 employees with the least vacation hours are:\n1. Job Title: Chief Financial Officer, Marital Status: M, Gender: F, Vacation Hours: 0\n2. Job Title: Production Technician - WC20, Marital Status: S, Gender: M, Vacation Hours: 0\n3. Job Title: Production Technician - WC50, Marital Status: S, Gender: M, Vacation Hours: 0\n4. Job Title: Production Technician - WC50, Marital Status: M, Gender: M, Vacation Hours: 1\n5. Job Title: Vice President of Engineering, Marital Status: S, Gender: F, Vacation Hours: 1'

In [129]:
# TC4 : (temperature = 0) | FAIL
# TC4 : (temperature = 0.2) | FAIL (salary_type is not a column, value 'salaried' is not a valid input)
chain.run("Get me the number of employees that are salaried from the prod.employee table")



> Entering new SQLDatabaseChain chain...
Get me the number of employees that are salaried from the prod.employee table
SQLQuery:SELECT COUNT(*) 
FROM prod.employee 
WHERE salary_type = 'salaried';

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "salary_type" does not exist
LINE 3: WHERE salary_type = 'salaried';
              ^

[SQL: SELECT COUNT(*) 
FROM prod.employee 
WHERE salary_type = 'salaried';]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [130]:
# TC5 : (temperature = 0) PASS
# TC5 : (temperature = 0.2) PASS
chain.run("how many males and females employees are there in the prod.employee table?")



> Entering new SQLDatabaseChain chain...
how many males and females employees are there in the prod.employee table?
SQLQuery:SELECT COUNT(*) FILTER (WHERE gender = 'M') AS males, COUNT(*) FILTER (WHERE gender = 'F') AS females
FROM prod.employee;
SQLResult: [(206, 84)]
Answer:There are 206 male employees and 84 female employees in the prod.employee table.
> Finished chain.


'There are 206 male employees and 84 female employees in the prod.employee table.'

In [131]:
# TC6 : (temperature = 0) FAIL (it had the right idea but failed due to incorrect column name reference. Hallucinating column names?)
# TC6 : (temperature = 0.2)
chain.run("Get all the employee records for the 10 oldest employees from the prod.employee table and calculate their individual age.")



> Entering new SQLDatabaseChain chain...
Get all the employee records for the 10 oldest employees from the prod.employee table and calculate their individual age.
SQLQuery:SELECT "employee_id", "first_name", "last_name", "DATE_PART('year', CURRENT_DATE) - DATE_PART('year', "birth_date") AS age
FROM "prod"."employee"
ORDER BY "birth_date" ASC
LIMIT 10;

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "") AS age
FROM ""
LINE 1: ...ear', CURRENT_DATE) - DATE_PART('year', "birth_date") AS age
                                                              ^

[SQL: SELECT "employee_id", "first_name", "last_name", "DATE_PART('year', CURRENT_DATE) - DATE_PART('year', "birth_date") AS age
FROM "prod"."employee"
ORDER BY "birth_date" ASC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [132]:
# TC7 : (temperature = 0)  | FAIL
# TC7 : (temperature = 0.2)  | FAIL (Employees is not a table, StartDate is not a column)
chain.run("How many years has the employee with the job title Chief Executive Officer been working at the company?")



> Entering new SQLDatabaseChain chain...
How many years has the employee with the job title Chief Executive Officer been working at the company?
SQLQuery:SELECT EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM "StartDate") AS years_working
FROM "Employees"
WHERE "JobTitle" = 'Chief Executive Officer'
LIMIT 1;

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "Employees" does not exist
LINE 2: FROM "Employees"
             ^

[SQL: SELECT EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM "StartDate") AS years_working
FROM "Employees"
WHERE "JobTitle" = 'Chief Executive Officer'
LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [134]:
# TC8 : (temperature = 0)  | FAIL (Can't seem to discern jobtile from "job title"
# so it's converting columns to camel case. It got the context correctly. salary_type is not a column, employees is not a table)
# TC8 : (temperature = 0.2)  | FAIL 
chain.run("How many employees that have the term Marketing in the job title are salaried?")



> Entering new SQLDatabaseChain chain...
How many employees that have the term Marketing in the job title are salaried?
SQLQuery:SELECT COUNT(*) 
FROM employees 
WHERE job_title LIKE '%Marketing%' AND salary_type = 'Salaried'
LIMIT 5;

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "employees" does not exist
LINE 2: FROM employees 
             ^

[SQL: SELECT COUNT(*) 
FROM employees 
WHERE job_title LIKE '%%Marketing%%' AND salary_type = 'Salaried'
LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [135]:

# It passes TC8 when I give it a ton of assistance
chain.run("How many employees (in the prod.employee table) whose jobtitle contain the term 'Marketing' are salaried. The salariedflag=True when an employee is salaried, else, false ")



> Entering new SQLDatabaseChain chain...
How many employees (in the prod.employee table) whose jobtitle contain the term 'Marketing' are salaried. The salariedflag=True when an employee is salaried, else, false 
SQLQuery:SELECT COUNT(*) 
FROM prod.employee 
WHERE jobtitle LIKE '%Marketing%' 
AND salariedflag = true;
SQLResult: [(1,)]
Answer:Final answer here: There is 1 employee in the prod.employee table whose jobtitle contains the term 'Marketing' and is salaried.
> Finished chain.


"Final answer here: There is 1 employee in the prod.employee table whose jobtitle contains the term 'Marketing' and is salaried."

In [136]:
# TC9: (temperature = 0)  (FAIL: it completely forgot prior tables referenced and made up two columns (job_title, salary_type) in its search)
# TC9: (temperature = 0.2) 
chain.run("How many employees that have the term 'Marketing' in their job title are not salaried?")



> Entering new SQLDatabaseChain chain...
How many employees that have the term 'Marketing' in their job title are not salaried?
SQLQuery:SELECT COUNT(*) 
FROM employees 
WHERE job_title LIKE '%Marketing%' 
AND salary_type <> 'Salaried';

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "employees" does not exist
LINE 2: FROM employees 
             ^

[SQL: SELECT COUNT(*) 
FROM employees 
WHERE job_title LIKE '%%Marketing%%' 
AND salary_type <> 'Salaried';]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [137]:
# It passed when I provided contextual assistance to the prompt
chain.run("How many employees (in the prod.employee tbale) that have the term 'Marketing' in their job title are not salaried? The salariedflag=True when an employee is salaried, else, false")



> Entering new SQLDatabaseChain chain...
How many employees (in the prod.employee tbale) that have the term 'Marketing' in their job title are not salaried? The salariedflag=True when an employee is salaried, else, false
SQLQuery:SELECT COUNT(*) 
FROM prod.employee 
WHERE jobtitle LIKE '%Marketing%' AND salariedflag = false;
SQLResult: [(8,)]
Answer:Final answer: There are 8 employees in the prod.employee table that have the term 'Marketing' in their job title and are not salaried.
> Finished chain.


"Final answer: There are 8 employees in the prod.employee table that have the term 'Marketing' in their job title and are not salaried."

In [138]:
# TC10 : (temperature = 0)  FAIL (Referenced the employees table instead of the employee table, and employeetype column which doesn't exist)
# TC10 : (temperature = 0.2) FAIL (EmployeeType is not a column)
chain.run("How many vacationhours, in days, do salaried employees with the highest vacation hours have? Limit to the top 10 records")



> Entering new SQLDatabaseChain chain...
How many vacationhours, in days, do salaried employees with the highest vacation hours have? Limit to the top 10 records
SQLQuery:SELECT "Employee"."VacationHours" / 8 AS "VacationDays"
FROM "Employee"
WHERE "Employee"."EmployeeType" = 'Salaried'
ORDER BY "Employee"."VacationHours" DESC
LIMIT 10;

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "Employee" does not exist
LINE 2: FROM "Employee"
             ^

[SQL: SELECT "Employee"."VacationHours" / 8 AS "VacationDays"
FROM "Employee"
WHERE "Employee"."EmployeeType" = 'Salaried'
ORDER BY "Employee"."VacationHours" DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [139]:
# It passes when I gave it contextual assistance and object references
# failed for on 0.2 model temp. "VacationHours" is not a column, vacationhours is, "SalariedFlag" is not a column,
chain.run("How many vacationhours, in days, do salaried employees (from the prod.employee table) with the highest vacationhours have? Limit to the top 10 records. The salariedflag=True when an employee is salaried, else, false")



> Entering new SQLDatabaseChain chain...
How many vacationhours, in days, do salaried employees (from the prod.employee table) with the highest vacationhours have? Limit to the top 10 records. The salariedflag=True when an employee is salaried, else, false
SQLQuery:SELECT "VacationHours" / 8 AS "VacationDays"
FROM prod.employee
WHERE "SalariedFlag" = true
ORDER BY "VacationHours" DESC
LIMIT 10;

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "VacationHours" does not exist
LINE 1: SELECT "VacationHours" / 8 AS "VacationDays"
               ^
HINT:  Perhaps you meant to reference the column "employee.vacationhours".

[SQL: SELECT "VacationHours" / 8 AS "VacationDays"
FROM prod.employee
WHERE "SalariedFlag" = true
ORDER BY "VacationHours" DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [140]:
# TC11 : (temperature = 0)  | FAIL (Column Salaried_Type and Vacation_hours don't exist, neither does employees table)
# TC11 : (temperature = 0.2) | FAIL (emmployees is not a table. However, It recognized SalariedFlag as a column)
chain.run("How many vacation hours, in days, do the top 10 not salaried employees with the highest vacation hours individually have? ")



> Entering new SQLDatabaseChain chain...
How many vacation hours, in days, do the top 10 not salaried employees with the highest vacation hours individually have? 
SQLQuery:SELECT "EmployeeID", "VacationHours" / 8 AS "VacationDays"
FROM "Employees"
WHERE "SalariedFlag" = false
ORDER BY "VacationHours" DESC
LIMIT 10;

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "Employees" does not exist
LINE 2: FROM "Employees"
             ^

[SQL: SELECT "EmployeeID", "VacationHours" / 8 AS "VacationDays"
FROM "Employees"
WHERE "SalariedFlag" = false
ORDER BY "VacationHours" DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [141]:
# passed with additional contextual assistance 
# failed with temp 0.2 as it continues to add quotation marks around column, schema, and tables names
chain.run("How many vacationhours, in days, do the not salaried employees (in the prod.employee table) with the highest vacation hours individually have?The salariedflag=True when an employee is salaried, else, false. Limit to the top 10 records ")



> Entering new SQLDatabaseChain chain...
How many vacationhours, in days, do the not salaried employees (in the prod.employee table) with the highest vacation hours individually have?The salariedflag=True when an employee is salaried, else, false. Limit to the top 10 records 
SQLQuery:SELECT "VacationHours" / 8 AS "VacationDays"
FROM "prod"."employee"
WHERE "SalariedFlag" = false
ORDER BY "VacationHours" DESC
LIMIT 10;

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "VacationHours" does not exist
LINE 1: SELECT "VacationHours" / 8 AS "VacationDays"
               ^
HINT:  Perhaps you meant to reference the column "employee.vacationhours".

[SQL: SELECT "VacationHours" / 8 AS "VacationDays"
FROM "prod"."employee"
WHERE "SalariedFlag" = false
ORDER BY "VacationHours" DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [142]:
# TC12 : (temperature = 0)  | FAIL
# TC12 : (temperature = 0.2)  | FAIL (Job title is not a column, Salary Type is not a column. Used incorrect syntax and broke SQL rule)
chain.run("How many vacation hours, in days, do the top 10 not salaried employees with the highest vacation hours individually have? Only return the Job title and vacation days")



> Entering new SQLDatabaseChain chain...
How many vacation hours, in days, do the top 10 not salaried employees with the highest vacation hours individually have? Only return the Job title and vacation days
SQLQuery:SELECT "Job title", "Vacation hours" / 8 AS "Vacation days"
FROM employees
WHERE "Salary type" != 'Salaried'
ORDER BY "Vacation hours" DESC
LIMIT 10;

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "employees" does not exist
LINE 2: FROM employees
             ^

[SQL: SELECT "Job title", "Vacation hours" / 8 AS "Vacation days"
FROM employees
WHERE "Salary type" != 'Salaried'
ORDER BY "Vacation hours" DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [143]:
#  test case passed
# test case failed with temp @ 0.2, added quotations around all db objects
chain.run("How many vacationhours, in days, do non-salaried employees (in the prod.employee table) with the highest vacationhours individually have?The salariedflag=True when an employee is salaried, else, false. Only return the Job title and vacation day and Limit to the top 10 records")



> Entering new SQLDatabaseChain chain...
How many vacationhours, in days, do non-salaried employees (in the prod.employee table) with the highest vacationhours individually have?The salariedflag=True when an employee is salaried, else, false. Only return the Job title and vacation day and Limit to the top 10 records
SQLQuery:SELECT "JobTitle", "VacationHours"/8 AS "VacationDays"
FROM prod.employee
WHERE "SalariedFlag" = false
ORDER BY "VacationHours" DESC
LIMIT 10;

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "JobTitle" does not exist
LINE 1: SELECT "JobTitle", "VacationHours"/8 AS "VacationDays"
               ^
HINT:  Perhaps you meant to reference the column "employee.jobtitle".

[SQL: SELECT "JobTitle", "VacationHours"/8 AS "VacationDays"
FROM prod.employee
WHERE "SalariedFlag" = false
ORDER BY "VacationHours" DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [144]:
# TC13 : (temperature = 0)  | FAIL (referenced employees table which doesn't exist 
# instead of referencing employeedepartmenthistory and department Tables.Department_Group_Name should groupname)

# TC13 : (temperature = 0.2)  | FAIL (referenced departments table which doesn't exist instead of referencing employeedepartmenthistory 
# and department Tables. Department_Group_Name should groupname. Added quotations on db objects)
chain.run("How many employees does each department have? Return the department group name and employee count for each department group name")



> Entering new SQLDatabaseChain chain...
How many employees does each department have? Return the department group name and employee count for each department group name
SQLQuery:SELECT "department_group_name", COUNT(*) AS "employee_count"
FROM "departments"
GROUP BY "department_group_name"

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "departments" does not exist
LINE 2: FROM "departments"
             ^

[SQL: SELECT "department_group_name", COUNT(*) AS "employee_count"
FROM "departments"
GROUP BY "department_group_name"]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [145]:
#  Passed when I add contextual assistance
chain.run("How many distinct employees (businessentityid from the prod.employeedepartmenthistory table) does each department have? Return the department groupname (from the prod.department table) and employee count for each department groupname")



> Entering new SQLDatabaseChain chain...
How many distinct employees (businessentityid from the prod.employeedepartmenthistory table) does each department have? Return the department groupname (from the prod.department table) and employee count for each department groupname
SQLQuery:SELECT d.groupname, COUNT(DISTINCT edh.businessentityid) AS employee_count
FROM prod.department d
JOIN prod.employeedepartmenthistory edh ON d.departmentid = edh.departmentid
GROUP BY d.groupname
ORDER BY employee_count DESC;
SQLResult: [('Manufacturing', 185), ('Executive General and Administration', 35), ('Sales and Marketing', 28), ('Inventory Management', 19), ('Research and Development', 14), ('Quality Assurance', 12)]
Answer:Final answer: Each department has the following number of distinct employees:
- Manufacturing: 185
- Executive General and Administration: 35
- Sales and Marketing: 28
- Inventory Management: 19
- Research and Development: 14
- Quality Assurance: 12
> Finished chain.


'Final answer: Each department has the following number of distinct employees:\n- Manufacturing: 185\n- Executive General and Administration: 35\n- Sales and Marketing: 28\n- Inventory Management: 19\n- Research and Development: 14\n- Quality Assurance: 12'

In [146]:
# TC14 : (temperature = 0) | FAIL (departments is not a table, departments_group_name is not a column)
# TC14 : (temperature = 0.2) | FAIL (added quotations on all db objects. employees is not a table, is_active is not a column
# departments is not a table)
chain.run("How many active employees does each department have? Return the department group name and employee count for each department group name")



> Entering new SQLDatabaseChain chain...
How many active employees does each department have? Return the department group name and employee count for each department group name
SQLQuery:SELECT "department_group_name", COUNT(*) AS "employee_count"
FROM "departments"
JOIN "employees" ON "departments"."department_id" = "employees"."department_id"
WHERE "employees"."is_active" = true
GROUP BY "department_group_name"

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "departments" does not exist
LINE 2: FROM "departments"
             ^

[SQL: SELECT "department_group_name", COUNT(*) AS "employee_count"
FROM "departments"
JOIN "employees" ON "departments"."department_id" = "employees"."department_id"
WHERE "employees"."is_active" = true
GROUP BY "department_group_name"]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [147]:
# Passed with contextual assistance 
chain.run("How many distinct active employees (businessentityid from the prod.employeedepartmenthistory table) does each department (from the prod.department table) have? Return the department groupname and employee count for each department group name")



> Entering new SQLDatabaseChain chain...
How many distinct active employees (businessentityid from the prod.employeedepartmenthistory table) does each department (from the prod.department table) have? Return the department groupname and employee count for each department group name
SQLQuery:SELECT d.groupname, COUNT(DISTINCT edh.businessentityid) AS employee_count
FROM prod.department d
JOIN prod.employeedepartmenthistory edh ON d.departmentid = edh.departmentid
WHERE edh.enddate IS NULL
GROUP BY d.groupname
ORDER BY employee_count DESC;
SQLResult: [('Manufacturing', 185), ('Executive General and Administration', 35), ('Sales and Marketing', 27), ('Inventory Management', 18), ('Research and Development', 14), ('Quality Assurance', 11)]
Answer:Final answer: Each department has the following number of distinct active employees:
- Manufacturing: 185
- Executive General and Administration: 35
- Sales and Marketing: 27
- Inventory Management: 18
- Research and Development: 14
- Quality As

'Final answer: Each department has the following number of distinct active employees:\n- Manufacturing: 185\n- Executive General and Administration: 35\n- Sales and Marketing: 27\n- Inventory Management: 18\n- Research and Development: 14\n- Quality Assurance: 11'

In [148]:
# TC15 : (temperature = 0)  | FAIL (employee_id is not a column, shift_type is not a name, status is not a column, 
# and employees is not a table)

# TC15 : (temperature = 0.2)  | FAIL (employee_id is not a column, shift_type is not a name, status is not a column, 
# and employees is not a table. Added quotation on all db objects)
chain.run(" How many distinct active employees work in each shift type? Return the count and shift type only")



> Entering new SQLDatabaseChain chain...
 How many distinct active employees work in each shift type? Return the count and shift type only
SQLQuery:SELECT COUNT(DISTINCT "employee_id"), "shift_type"
FROM "employees"
WHERE "is_active" = true
GROUP BY "shift_type"

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "employees" does not exist
LINE 2: FROM "employees"
             ^

[SQL: SELECT COUNT(DISTINCT "employee_id"), "shift_type"
FROM "employees"
WHERE "is_active" = true
GROUP BY "shift_type"]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [149]:
# Passed with contextual Assistance
chain.run(" How many distinct active employees (businessentityid from the prod.employeedepartmenthistory table) work in each shift type (name column from the prod.shift table)? Return the count and shift type only")



> Entering new SQLDatabaseChain chain...
 How many distinct active employees (businessentityid from the prod.employeedepartmenthistory table) work in each shift type (name column from the prod.shift table)? Return the count and shift type only
SQLQuery:SELECT COUNT(DISTINCT edh.businessentityid), s.name
FROM prod.employeedepartmenthistory edh
JOIN prod.shift s ON edh.shiftid = s.shiftid
WHERE edh.enddate IS NULL
GROUP BY s.name
SQLResult: [(176, 'Day'), (62, 'Evening'), (52, 'Night')]
Answer:There are 176 distinct active employees working in the "Day" shift, 62 distinct active employees working in the "Evening" shift, and 52 distinct active employees working in the "Night" shift.
> Finished chain.


'There are 176 distinct active employees working in the "Day" shift, 62 distinct active employees working in the "Evening" shift, and 52 distinct active employees working in the "Night" shift.'

In [150]:
# TC16 : (temperature = 0)  | FAIL (no table called departments, no table called employees, no table called shifts 
# shift_type is not a column, status is not a column, employee_id is not a column)

# TC16 : (temperature = 0.2)  | FAIL (no table called departments, no table called employees, no table called shifts 
# shift_type is not a column, status is not a column, employee_id is not a column, end_time is not a column. 
# Input value case sensitivity issue. Added quotations on all db objects. Unnecessary logic inclusion: "end_time > CURRENT_DATE")
chain.run("Which departments have active employees that work the night shift and how many employees are there?")



> Entering new SQLDatabaseChain chain...
Which departments have active employees that work the night shift and how many employees are there?
SQLQuery:SELECT "department_name", COUNT(*) 
FROM "departments" 
JOIN "employees" ON "departments"."department_id" = "employees"."department_id" 
JOIN "shifts" ON "employees"."employee_id" = "shifts"."employee_id" 
WHERE "shifts"."shift_name" = 'night' 
AND "shifts"."end_time" > CURRENT_DATE 
GROUP BY "department_name"

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "departments" does not exist
LINE 2: FROM "departments" 
             ^

[SQL: SELECT "department_name", COUNT(*) 
FROM "departments" 
JOIN "employees" ON "departments"."department_id" = "employees"."department_id" 
JOIN "shifts" ON "employees"."employee_id" = "shifts"."employee_id" 
WHERE "shifts"."shift_name" = 'night' 
AND "shifts"."end_time" > CURRENT_DATE 
GROUP BY "department_name"]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [151]:
# FAIL: issue with case sensitivity with the 'night' value even when I added a note in parenthesis regarding case sensitivity
# Also, didn't include logic for active employee which is to ensure enddate is not null. 
# in 0.2 temp added incorrect logic for Active employee: s.starttime > '00:00:00' AND s.endtime < '06:00:00' 
chain.run("Which departments (groupname from the prod.department table) have active employees that work at night(case sensitive) (name from the prod.shift table) and how many employees (businessentityid from the prod.employeedepartmenthistory table) are there?")



> Entering new SQLDatabaseChain chain...
Which departments (groupname from the prod.department table) have active employees that work at night(case sensitive) (name from the prod.shift table) and how many employees (businessentityid from the prod.employeedepartmenthistory table) are there?
SQLQuery:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Request failed due to server shutdown {
  "error": {
    "message": "Request failed due to server shutdown",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'Request failed due to server shutdown', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Fri, 25 Aug 2023 12:38:09 GMT', 'Content-Type': 'application/json', 'Content-Length': '141', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-model': 'gpt-3.5-turbo-16k-0613', 'openai-organization': 'user-bxpelthnfhjktajxwu3m1bns', 'openai-processing-ms': '13932', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'x-ratelimit-limit-requests': '3500', 'x-ratelimit-limit-tokens': '180000', 'x-ratelimit-remaining-requests': '3499', 'x-ratelimit-remaining-tokens': '179643', 

SELECT d.groupname, COUNT(edh.businessentityid) 
FROM prod.department d 
JOIN prod.employeedepartmenthistory edh ON d.departmentid = edh.departmentid 
JOIN prod.shift s ON edh.shiftid = s.shiftid 
WHERE s.name = 'night' AND s.starttime > '00:00:00' AND s.endtime < '06:00:00' 
GROUP BY d.groupname
SQLResult: []
Answer:Final answer here: There are no departments with active employees that work at night.
> Finished chain.


'Final answer here: There are no departments with active employees that work at night.'

In [152]:
# Passes with more additional contextual assistance. Letting it know that 
# filtering values may be case sensitive prompted it to use the ILIKE function to adjust for all case sensitive scenarios

# in 0.2 temp, failed. Added incorrect Active employye logic: edh."enddate" > CURRENT_DATE. unnecessary quotations on db objects.
chain.run("Which departments (groupname from the prod.department table) have active (enddate in the prod.employeedepartmenthistory table) employees that work at night (name from the prod.shift table) and how many employees (businessentityid from the prod.employeedepartmenthistory table) are there? Filtering condition may require you to handle case sensitivity")



> Entering new SQLDatabaseChain chain...
Which departments (groupname from the prod.department table) have active (enddate in the prod.employeedepartmenthistory table) employees that work at night (name from the prod.shift table) and how many employees (businessentityid from the prod.employeedepartmenthistory table) are there? Filtering condition may require you to handle case sensitivity
SQLQuery:SELECT d."groupname", COUNT(edh."businessentityid") AS num_employees
FROM prod.department d
JOIN prod.employeedepartmenthistory edh ON d."departmentid" = edh."departmentid"
JOIN prod.shift s ON edh."shiftid" = s."shiftid"
WHERE edh."enddate" > CURRENT_DATE
AND LOWER(s."name") LIKE '%night%'
GROUP BY d."groupname"
SQLResult: []
Answer:Final answer here: There are no departments with active employees that work at night.
> Finished chain.


'Final answer here: There are no departments with active employees that work at night.'